In [ ]:
#used the https://jikan.moe/#how-it-works, and manual entry to insert data
import requests
import psycopg2
import json

In [ ]:
#establishing the connection
conn = psycopg2.connect(
    database='',
    user='',
    password='',
    host='',
    port=''
)

In [4]:
#creating cursor object
cursor = conn.cursor()

In [168]:
#generate a secure, deterministic token from a string using SHA-256
import hashlib

def generate_token_from_string(input_string):
    hash_object = hashlib.sha256(input_string.encode('utf-8'))
    return hash_object.hexdigest()

In [ ]:
dummy_user_token = generate_token_from_string("DUMMY123")
dummy_user_token

In [ ]:
#when query fails perform rollback
conn.rollback()


In [ ]:
#✅ add user
# try:
#     cursor.execute(
#         "INSERT INTO weebyomiuri_user (username, email, phone, security_token) VALUES (%s, %s, %s, %s)",
#         ("dummy", "dummy@gmail.com", '000-000-0000', dummy_user_token)
#     )
#     conn.commit()
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()


In [ ]:
#✅ give admin and add permissions
# try:
#     cursor.execute(
#         "INSERT INTO admin (userID, admin_level, permissions) VALUES (%s, %s, %s)",
#         (1, "SUPER ADMIN", ['READ','CREATE', 'EDIT', 'DELETE'])
#     )
#     conn.commit()
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()


In [ ]:
#✅ add records to the premiered_season_year table
# try:
#     season_year= [('SPRING', 2025), ('WINTER', 2025), ('FALL', 2024), ('SUMMER', 2024),('SPRING', 2024), ('WINTER', 2024)]
#     insert_season_year = """INSERT INTO premiered_season_year (season,year) VALUES (%s, %s)"""
#     cursor.executemany(insert_season_year, season_year)
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()

In [ ]:

#✅ insert genre
# url = "https://api.jikan.moe/v4/genres/anime" 
# headers = {'Content-Type': 'application/json'}
# try:
#   response = requests.get(url, headers=headers)
#   response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
  
# except requests.exceptions.RequestException as e:
#   print(f"An error occurred: {e}")

# data = response.json()['data']
# genres_list = []
# for i in data:
#     if i['name'] != 'Hentai':
#         genres_list.append(i['name'])
# genres_list = list(zip(genres_list))

# insert_genre = """INSERT INTO genre (genre_name) VALUES (%s)"""
# cursor.executemany(insert_genre, genres_list)


In [32]:
#✅ get anime data
url = "https://api.jikan.moe/v4/top/anime?sfw&page=1"
headers = {'Content-Type': 'application/json'}

try:
  response = requests.get(url, headers=headers)
  response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
  
except requests.exceptions.RequestException as e:
  print(f"An error occurred: {e}")

anime_data = response.json()

In [66]:
#✅ get manga data
url = "https://api.jikan.moe/v4/top/manga?page=1"
headers = {'Content-Type': 'application/json'}

try:
  response = requests.get(url, headers=headers)
  response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
  
except requests.exceptions.RequestException as e:
  print(f"An error occurred: {e}")

manga_data = response.json()

In [ ]:
#print(json.dumps(manga_data['data'][0], indent=4))
#print(manga_data['data'][0]['title_english'])
#print(manga_data['data'][0]['title_japanese'])
# print(manga_data['data'][0]['synopsis'])
# if (manga_data['data'][2]['volumes'] != None):
#      print("TEST NONE")
# print(manga_data['data'][0]['volumes'])
# print(manga_data['data'][0]['published']['from'])
# print(manga_data['data'][0]['published']['to'])
# print(manga_data['data'][10]['serializations'])
# if (len(manga_data['data'][10]['serializations']) == 0):
#      print('tr')
# print(manga_data['data'][0]['images']['jpg']['image_url'])
# print(manga_data['data'][0]['url'])

# english_title VARCHAR(150),
# japanese_title VARCHAR(150),
# manga_synopsis TEXT,
# volumes INTEGER,
# publish_start_date DATE,
# publish_end_date DATE,
# serialized BOOLEAN,
# manga_image VARCHAR(255),
# sources_manga_analysis VARCHAR(255)

#✅ insert manga table base
# insert_manga_query = "INSERT into manga(english_title, japanese_title, manga_synopsis, volumes" \
# ",publish_start_date, publish_end_date, serialized, sources_manga_analysis, manga_image) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

# try:
#     for i in range(1, len(manga_data['data'])):
#         if(manga_data['data'][i]['title_english'] != None):
#             if(manga_data['data'][i]['volumes'] != None):
#                 if(len(manga_data['data'][i]['serializations']) == 0):
#                     manga_data_noser = (manga_data['data'][i]['title_english'],manga_data['data'][i]['title_japanese'],manga_data['data'][i]['synopsis'],
#                     manga_data['data'][i]['volumes'],manga_data['data'][i]['published']['from'], manga_data['data'][i]['published']['to'],
#                     'FALSE', manga_data['data'][i]['url'], manga_data['data'][i]['images']['jpg']['image_url'])
#                     cursor.execute(insert_manga_query,manga_data_noser)
#                 else:
#                     manga_data_withserial =(manga_data['data'][i]['title_english'],manga_data['data'][i]['title_japanese'],manga_data['data'][i]['synopsis'],
#                     manga_data['data'][i]['volumes'],manga_data['data'][i]['published']['from'], manga_data['data'][i]['published']['to'],
#                     'TRUE', manga_data['data'][i]['url'],manga_data['data'][i]['images']['jpg']['image_url'])
#                     cursor.execute(insert_manga_query,manga_data_withserial)
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()


In [27]:
#✅ get more detailed anime data
id = anime_data['data'][0]['mal_id']
url = f"https://api.jikan.moe/v4/anime/{id}/full"
headers = {'Content-Type': 'application/json'}
url_recommendations = f"https://api.jikan.moe/v4/anime/{id}/recommendations"

try:
  response = requests.get(url, headers=headers)
  response_recommendations = requests.get(url_recommendations, headers=headers)
  response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
  
except requests.exceptions.RequestException as e:
  print(f"An error occurred: {e}")

data_anime_detailed = response.json()
data_anime_recommendations = response_recommendations.json()

In [28]:
data_anime_detailed['data']['episodes']

28

In [30]:
data_anime_recommendations['data'][3]['entry']['title']

'Dungeon Meshi'

In [ ]:
#✅ testing code
# for i in range(1, len(anime_data['data'])):
#     print(data['data'][i]['mal_id'])


# print(anime_data['data'][0]['synopsis'])
# print(anime_data['data'][0]['broadcast']['string'])
# print(anime_data['data'][0]['aired']['from'])
# print(anime_data['data'][0]['aired']['to'])
# print(anime_data['data'][0]['season'].upper())
# print(anime_data['data'][0]['year'])
# print(anime_data['data'][0]['url'])
# print(anime_data['data'][0]['images']['jpg']['image_url'])


2024-03-22T00:00:00+00:00


In [ ]:
#✅ insert anime table base
# insert_anime_query = "INSERT into anime(english_title, japanese_title, anime_synopsis,broadcast_date_time " \
# ",release_date,end_date, sources_anime_analysis, anime_image) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# try:
#     for i in range(1, len(anime_data['data'])):
#         cursor.execute(insert_anime_query,
#             (anime_data['data'][i]['title_english'],anime_data['data'][i]['title_japanese'], anime_data['data'][i]['synopsis'],anime_data['data'][i]['broadcast']['string'],
#             anime_data['data'][i]['aired']['from'],anime_data['data'][i]['aired']['to'],anime_data['data'][i]['url'],anime_data['data'][i]['images']['jpg']['image_url']))
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()

In [ ]:
#✅ clean the data
# try: 
#     cursor.execute("UPDATE manga SET publish_end_date = %s WHERE mangaID = %s", ('1973-05-13',12))
#     conn.commit()
#     # cursor.execute("DELETE FROM manga WHERE mangaID = %s", (1,))
#     # conn.commit()
# except Exception as e:
#     print("Error:", e)
#     conn.rollback()


In [ ]:
#✅ query to get table by name
search_tags = 'select * from manga;'
#executing query
cursor.execute(search_tags)
#fetching the result
cursor.fetchall()

In [138]:
#✅ commit your changes in the database
conn.commit()

In [251]:
#✅ close connection
conn.close()